In [2]:
with open('day14-input.txt') as f:
    txt = f.read().splitlines()
    
txt[:20]

['mask = 11110100010101111011001X0100XX00100X',
 'mem[17610] = 1035852',
 'mem[55284] = 229776690',
 'mem[16166] = 12685380',
 'mem[8340] = 16011',
 'mask = 0X1X0X010101011X10X101000X0001110100',
 'mem[968] = 15992',
 'mem[32758] = 7076',
 'mem[30704] = 1701',
 'mem[33719] = 58012',
 'mem[20818] = 25927237',
 'mem[16718] = 46485',
 'mask = 111001111X0X0X101X111X1X001XX0011010',
 'mem[2115] = 14848',
 'mem[42753] = 617',
 'mem[56076] = 9933868',
 'mem[19106] = 43503',
 'mem[10073] = 32909',
 'mem[40830] = 1959686',
 'mask = X11X00000XX1011X10000X01110000X0001X']

In [37]:
def apply_mask( n, mask ):
    one_mask = int(re.sub("X", "0", mask), 2)
    zero_mask = int(re.sub("X", "1", mask), 2)
    return (n | one_mask) & zero_mask

In [45]:
import re

mask = None
mem = {}

for l in txt:
    if re.match("^mask", l):
        mask = re.match("mask = ([10X]+)", l).group(1)
    else:
        m = re.match("mem\[(\d+)\] = (\d+)", l)
        addr = int( m.group(1) )
        num = int( m.group(2) )
        mem[addr] = apply_mask( num, mask )
        
sum( mem.values() )

10885823581193

# Part 2

In [283]:
test = """mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1""".splitlines()

In [216]:
zero_base_mask = 2 ** 36 - 1
mask = 'X1001X'

`zeros_ones`: for a set of positions for `X` in the mask, create a list of permutations for all possible 1 and 0 values for those Xs as a list of pairs of sets of indicies.

In [263]:
def xpos( mask ):
    return {2 ** i for (i, c) in enumerate(mask[::-1]) if c == 'X'}

[bin(n) for n in xpos( 'X1001X')]

['0b100000', '0b1']

In [199]:
from itertools import chain, combinations

def zeros_ones_perms(s):
    return [(set(i), s.difference(i)) for i in chain.from_iterable(combinations(s, r) for r in range(len(s)+1))]

In [212]:
zeros_ones_perms( xpos( 'X1001X' ))

[(set(), {1, 32}), ({32}, {1}), ({1}, {32}), ({1, 32}, set())]

In [297]:
def apply_mask( mask, n ):
    # The ones and zeros can just be applied as a bitwise `OR` operation. Doesn't matter what we set the `X`s to.
    or_mask = int(re.sub('X', '0', mask), 2)
    omasked = n | or_mask
    
    # Find the positions of Xs
    float_pos = xpos( mask )
    # Get a list of pairs of zero indicies and one indicies representing
    # every permutation of ones or zeros subbed for X
    zeros_ones = zeros_ones_perms( float_pos )
    
    # Replace the the number with a list of binary digits as strings (for easier manipulation)
    ns = []
    
    for (zeros, ones) in zeros_ones:
        n_ = omasked & (zero_base_mask ^ sum(zeros)) | sum(ones)
        ns.append(n_)
        
    return ns

mask = 'X1001X'
n = 42

apply_mask(mask, 42)

[59, 27, 58, 26]

In [299]:
import re

mask = None
mem = {}

for l in txt:
    if re.match("^mask", l):
        mask = re.match("mask = ([10X]+)", l).group(1)
    else:
        m = re.match("mem\[(\d+)\] = (\d+)", l)
        addr = int( m.group(1) )
        num = int( m.group(2) )
        
        addrs = apply_mask( mask, addr )
        
        for addr in addrs:
            mem[addr] = num
        

sum(mem.values())

3816594901962